# SIT220 – Data Wrangling  
## Task 3 – pandas vs SQL

**Name:** Barbara Rein Peralta Calma 

**ID:** s224712709

**email:** s224712709@deakin.edu.au

**undergraduate**

This notebook uses the nycflights13 dataset, which contains details about all flights that departed in 2013 from the three major New York airports: EWR, JFK, and LGA.
The purpose of this task is to practise working with pandas and an SQLite database, and to learn how to write SQL-style queries using pandas only.

First, I load the CSV files into pandas, then save them into a small SQLite database on my computer.
For each SQL query in the task sheet, I:

Run the original SQL query on the SQLite database,

Write equivalent pandas code that produces the same result without using SQL,

Compare the two outputs using pd.testing.assert_frame_equal to check that they match (allowing for differences in row order when needed).

In [4]:
import pandas as pd
import sqlite3

"""
Data loading and SQLite setup
"""

flights = pd.read_csv("nycflights13_flights.csv", comment='#', skipinitialspace=True)
airlines = pd.read_csv("nycflights13_airlines.csv", comment='#', skipinitialspace=True)
airports = pd.read_csv("nycflights13_airports.csv", comment='#', skipinitialspace=True)
planes = pd.read_csv("nycflights13_planes.csv", comment='#', skipinitialspace=True)
weather = pd.read_csv("nycflights13_weather.csv", comment='#', skipinitialspace=True)

db_path = "nycflights13.sqlite"
conn = sqlite3.connect(db_path)

flights.to_sql("flights", conn, if_exists="replace", index=False)
airlines.to_sql("airlines", conn, if_exists="replace", index=False)
airports.to_sql("airports", conn, if_exists="replace", index=False)
planes.to_sql("planes", conn, if_exists="replace", index=False)
weather.to_sql("weather", conn, if_exists="replace", index=False)

print("SQLite database created at:", db_path)
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn))


SQLite database created at: nycflights13.sqlite
       name
0   flights
1  airlines
2  airports
3    planes
4   weather


In [87]:
"""
helper for comparing SQL and pandas outputs
"""

def compare_frames(sql_df, pandas_df, sort_by=None):
    import pandas as _pd
    if sort_by is not None:
        sql_df = sql_df.sort_values(sort_by).reset_index(drop=True)
        pandas_df = pandas_df.sort_values(sort_by).reset_index(drop=True)
    else:
        sql_df = sql_df.reset_index(drop=True)
        pandas_df = pandas_df.reset_index(drop=True)
    _pd.testing.assert_frame_equal(sql_df, pandas_df)
    return sql_df


---
### Query 1

```sql
SELECT DISTINCT engine FROM planes
```

This lists each different engine type that appears in the planes table.


In [7]:
# Query 1
task1_sql = pd.read_sql_query("""SELECT DISTINCT engine FROM planes""", conn)

task1_my = planes[["engine"]].drop_duplicates()
compare_frames(task1_sql, task1_my, sort_by=["engine"])


,engine
0,4 Cycle
1,Reciprocating
2,Turbo-fan
3,Turbo-jet
4,Turbo-prop
5,Turbo-shaft


---
### Query 2

```sql
SELECT DISTINCT type, engine FROM planes
```

This lists all unique (type, engine) combinations for planes.


In [9]:
# Query 2
task2_sql = pd.read_sql_query("""SELECT DISTINCT type, engine FROM planes""", conn)

task2_my = planes[["type", "engine"]].drop_duplicates()
compare_frames(task2_sql, task2_my, sort_by=["type", "engine"])


,type,engine
0,Fixed wing multi engine,Reciprocating
1,Fixed wing multi engine,Turbo-fan
2,Fixed wing multi engine,Turbo-jet
3,Fixed wing multi engine,Turbo-prop
4,Fixed wing single engine,4 Cycle
5,Fixed wing single engine,Reciprocating
6,Rotorcraft,Turbo-shaft


---
### Query 3

```sql
SELECT COUNT(*), engine FROM planes GROUP BY engine
```

Counts how many planes there are for each engine type.


In [11]:
# Query 3
task3_sql = pd.read_sql_query("""SELECT COUNT(*), engine FROM planes GROUP BY engine""", conn)

task3_my = (planes.groupby("engine", as_index=False)
            .size()
            .rename(columns={"size": "COUNT(*)"}))
task3_my = task3_my[["COUNT(*)", "engine"]]  # Match column order
compare_frames(task3_sql, task3_my, sort_by=["engine"])


,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


---
### Query 4

```sql
SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type
```

Counts planes for every combination of engine and type.


In [13]:
# Query 4
task4_sql = pd.read_sql_query("""SELECT COUNT(*), engine, type FROM planes GROUP BY engine, type""", conn)

task4_my = (planes.groupby(["engine", "type"], as_index=False)
            .size()
            .rename(columns={"size": "COUNT(*)"}))
task4_my = task4_my[["COUNT(*)", "engine", "type"]]  # Match column order
compare_frames(task4_sql, task4_my, sort_by=["engine", "type"])


,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


---
### Query 5

```sql
SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes GROUP BY engine, manufacturer
```

Summarises min, average and max manufacturing year for each engine/manufacturer combination.


In [15]:
# Query 5
task5_sql = pd.read_sql_query("""SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer FROM planes GROUP BY engine, manufacturer""", conn)

agg = (planes.groupby(["engine", "manufacturer"], as_index=False)["year"]
       .agg(["min", "mean", "max"]))
agg = agg.reset_index()
task5_my = agg.rename(columns={"min": "MIN(year)", "mean": "AVG(year)", "max": "MAX(year)"})
task5_my = task5_my[["MIN(year)", "AVG(year)", "MAX(year)", "engine", "manufacturer"]]  # Match column order
compare_frames(task5_sql, task5_my, sort_by=["engine", "manufacturer"])


,MIN(year),AVG(year),MAX(year),engine,manufacturer
0,1975.0,1975.000000,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.000000,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L
5,1959.0,1971.142857,1983.0,Reciprocating,CESSNA
6,2007.0,2007.000000,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,1959.000000,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,1956.000000,1956.0,Reciprocating,DOUGLAS
9,2007.0,2007.000000,2007.0,Reciprocating,FRIEDEMANN JON


---
### Query 6

```sql
SELECT * FROM planes WHERE speed IS NOT NULL
```

Keeps only planes where the speed column is not missing.


In [17]:
# Query 6
task6_sql = pd.read_sql_query("""SELECT * FROM planes WHERE speed IS NOT NULL""", conn)

task6_my = planes[planes["speed"].notna()]
compare_frames(task6_sql, task6_my)


,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


---
### Query 7

```sql
SELECT tailnum FROM planes WHERE seats BETWEEN 150 AND 210 AND year >= 2011
```

Tail numbers of planes with 150–210 seats and built from 2011 onwards.


In [19]:
# Query 7
task7_sql = pd.read_sql_query("""SELECT tailnum FROM planes WHERE seats BETWEEN 150 AND 210 AND year >= 2011""", conn)

mask = planes["seats"].between(150, 210) & (planes["year"] >= 2011)
task7_my = planes.loc[mask, ["tailnum"]]
compare_frames(task7_sql, task7_my)


,tailnum
0,N150UW
1,N151UW
2,N152UW
3,N153UW
4,N154UW
...,...
87,N851VA
88,N852VA
89,N853VA
90,N854VA


---
### Query 8

```sql
SELECT tailnum, manufacturer, seats FROM planes WHERE manufacturer IN ('BOEING', 'AIRBUS', 'EMBRAER') AND seats>390
```

Large Boeing, Airbus or Embraer planes with more than 390 seats.


In [21]:
# Query 8
task8_sql = pd.read_sql_query("""SELECT tailnum, manufacturer, seats FROM planes WHERE manufacturer IN ('BOEING', 'AIRBUS', 'EMBRAER') AND seats>390""", conn)

mask = planes["manufacturer"].isin(["BOEING", "AIRBUS", "EMBRAER"]) & (planes["seats"] > 390)
task8_my = planes.loc[mask, ["tailnum", "manufacturer", "seats"]]
compare_frames(task8_sql, task8_my)


,tailnum,manufacturer,seats
0,N206UA,BOEING,400
1,N228UA,BOEING,400
2,N272AT,BOEING,400
3,N57016,BOEING,400
4,N670US,BOEING,450
5,N77012,BOEING,400
6,N777UA,BOEING,400
7,N78003,BOEING,400
8,N78013,BOEING,400
9,N787UA,BOEING,400


---
### Query 9

```sql
SELECT DISTINCT year, seats FROM planes WHERE year >= 2012 ORDER BY year ASC, seats DESC
```

Unique (year, seats) for planes from 2012 on, ordered by year then seats desc.


In [23]:
# Query 9
task9_sql = pd.read_sql_query("""SELECT DISTINCT year, seats FROM planes WHERE year >= 2012 ORDER BY year ASC, seats DESC""", conn)

mask = planes["year"] >= 2012
task9_my = (planes.loc[mask, ["year", "seats"]]
            .drop_duplicates()
            .sort_values(["year", "seats"], ascending=[True, False]))
compare_frames(task9_sql, task9_my, sort_by=["year", "seats"])


,year,seats
0,2012.0,5
1,2012.0,20
2,2012.0,140
3,2012.0,149
4,2012.0,182
5,2012.0,191
6,2012.0,200
7,2012.0,222
8,2012.0,260
9,2012.0,377


---
### Query 10

```sql
SELECT DISTINCT year, seats FROM planes WHERE year >= 2012 ORDER BY seats DESC, year ASC
```

Same as query 9 but ordered by seats desc then year asc.


In [25]:
# Query 10
task10_sql = pd.read_sql_query("""SELECT DISTINCT year, seats FROM planes WHERE year >= 2012 ORDER BY seats DESC, year ASC""", conn)

mask = planes["year"] >= 2012
task10_my = (planes.loc[mask, ["year", "seats"]]
             .drop_duplicates()
             .sort_values(["seats", "year"], ascending=[False, True]))
compare_frames(task10_sql, task10_my, sort_by=["seats", "year"])


,year,seats
0,2012.0,5
1,2012.0,20
2,2013.0,20
3,2013.0,95
4,2012.0,140
5,2013.0,140
6,2012.0,149
7,2012.0,182
8,2013.0,182
9,2012.0,191


---
### Query 11

```sql
SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer
```

Counts large (seats > 200) planes per manufacturer.


In [27]:
# Query 11
task11_sql = pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer""", conn)

mask = planes["seats"] > 200
task11_my = (planes.loc[mask]
             .groupby("manufacturer", as_index=False)
             .size()
             .rename(columns={"size": "COUNT(*)"}))
compare_frames(task11_sql, task11_my, sort_by=["manufacturer"])


,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


---
### Query 12

```sql
SELECT manufacturer, COUNT(*) FROM planes GROUP BY manufacturer HAVING COUNT(*) > 10
```

Manufacturers with more than 10 planes total.


In [29]:
# Query 12
task12_sql = pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes GROUP BY manufacturer HAVING COUNT(*) > 10""", conn)

task12_my = (planes.groupby("manufacturer", as_index=False)
             .size()
             .rename(columns={"size": "COUNT(*)"}))
task12_my = task12_my[task12_my["COUNT(*)"] > 10]
compare_frames(task12_sql, task12_my, sort_by=["manufacturer"])


,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299
5,MCDONNELL DOUGLAS,120
6,MCDONNELL DOUGLAS AIRCRAFT CO,103
7,MCDONNELL DOUGLAS CORPORATION,14


---
### Query 13

```sql
SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10
```

Manufacturers that have more than 10 planes with over 200 seats.


In [31]:
# Query 13
task13_sql = pd.read_sql_query("""SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10""", conn)

mask = planes["seats"] > 200
task13_my = (planes.loc[mask]
             .groupby("manufacturer", as_index=False)
             .size()
             .rename(columns={"size": "COUNT(*)"}))
task13_my = task13_my[task13_my["COUNT(*)"] > 10]
compare_frames(task13_sql, task13_my, sort_by=["manufacturer"])


,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


---
### Query 14


```sql
SELECT manufacturer, COUNT(*) AS howmany FROM planes GROUP BY manufacturer ORDER BY howmany DESC LIMIT 10
```

Top 10 manufacturers by number of planes.


In [33]:
# Query 14
task14_sql = pd.read_sql_query("""SELECT manufacturer, COUNT(*) AS howmany FROM planes GROUP BY manufacturer ORDER BY howmany DESC LIMIT 10""", conn)

task14_my = (planes.groupby("manufacturer", as_index=False)
             .size()
             .rename(columns={"size": "howmany"})
             .sort_values("howmany", ascending=False)
             .head(10))
compare_frames(task14_sql, task14_my, sort_by=["howmany", "manufacturer"])


,manufacturer,howmany
0,CANADAIR,9
1,CESSNA,9
2,MCDONNELL DOUGLAS CORPORATION,14
3,MCDONNELL DOUGLAS AIRCRAFT CO,103
4,MCDONNELL DOUGLAS,120
5,EMBRAER,299
6,AIRBUS,336
7,BOMBARDIER INC,368
8,AIRBUS INDUSTRIE,400
9,BOEING,1630


---
### Query 15

```sql
SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum
```

Left join flights with planes on tailnum and add plane year, speed and seats.


In [35]:
# Query 15
task15_sql = pd.read_sql_query("""SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum""", conn)

merged15 = flights.merge(
    planes[["tailnum", "year", "speed", "seats"]],
    on="tailnum",
    how="left",
    suffixes=('', '_plane')
)
# After merge: year stays as is (from flights), year_plane is from planes
# speed and seats are unique to planes, so no suffix
merged15 = merged15.rename(columns={
    "year_plane": "plane_year",
    "speed": "plane_speed",
    "seats": "plane_seats"
})
task15_my = merged15[task15_sql.columns]
compare_frames(task15_sql, task15_my)


/var/folders/6r/b1mcv41x74zckfyvktdgbp1r0000gn/T/ipykernel_15250/2502032277.py:13: FutureWarning: Mismatched null-like values None and nan found. In a future version, pandas equality-testing functions (e.g. assert_frame_equal) will consider these not-matching and raise.
  _pd.testing.assert_frame_equal(sql_df, pandas_df)


,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,origin,dest,air_time,distance,hour,minute,time_hour,plane_year,plane_speed,plane_seats
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,1999.0,NaN,149.0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,1998.0,NaN,149.0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,1990.0,NaN,178.0
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,2012.0,NaN,200.0
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,1991.0,NaN,178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,...,JFK,DCA,NaN,213,14,55,2013-09-30 14:00:00,NaN,NaN,NaN
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,...,LGA,SYR,NaN,198,22,0,2013-09-30 22:00:00,NaN,NaN,NaN
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,...,LGA,BNA,NaN,764,12,10,2013-09-30 12:00:00,NaN,NaN,NaN
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,...,LGA,CLE,NaN,419,11,59,2013-09-30 11:00:00,NaN,NaN,NaN


---
### Query 16

```sql
SELECT planes.*, airlines.* FROM
(SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier=airlines.carrier
```

Planes that appear in the flights data, joined with their airlines.


In [37]:
# Query 16
task16_sql = pd.read_sql_query("""SELECT planes.*, airlines.* FROM
(SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier=airlines.carrier""", conn)

cartail = flights[["carrier", "tailnum"]].drop_duplicates()
merged16 = cartail.merge(planes, on="tailnum", how="inner")
merged16 = merged16.merge(airlines, on="carrier", how="inner")
plane_cols = planes.columns.tolist()
airline_cols = airlines.columns.tolist()
task16_my = merged16[plane_cols + airline_cols]
compare_frames(task16_sql, task16_my, sort_by=plane_cols + airline_cols)


,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,carrier,name
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.
...,...,...,...,...,...,...,...,...,...,...,...
3334,N997AT,2002.0,Fixed wing multi engine,BOEING,717-200,2,100,NaN,Turbo-fan,FL,AirTran Airways Corporation
3335,N997DL,1992.0,Fixed wing multi engine,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,2,142,NaN,Turbo-fan,DL,Delta Air Lines Inc.
3336,N998AT,2002.0,Fixed wing multi engine,BOEING,717-200,2,100,NaN,Turbo-fan,FL,AirTran Airways Corporation
3337,N998DL,1992.0,Fixed wing multi engine,MCDONNELL DOUGLAS CORPORATION,MD-88,2,142,NaN,Turbo-jet,DL,Delta Air Lines Inc.


All done ;))